In [8]:
import openmc
from openmc.deplete import Chain
from pandas import DataFrame
from openmc.mgxs import FissionXS, ArbitraryXS, EnergyGroups

In [9]:
model = openmc.Model.from_xml()
chain_file = '../openmc/tests/chain_simple.xml'
chain = Chain.from_xml(chain_file)
reactions = chain.reactions

groups = EnergyGroups((0,20e6))
reaction_domain=model.materials[0]
xs = {}
for rx in reactions:
    if rx == 'fission':
        xs[rx] = FissionXS(domain=reaction_domain,
                     energy_groups=groups, by_nuclide=True)
    else:
        xs[rx] = ArbitraryXS(rx, domain=reaction_domain,
                       energy_groups=groups, by_nuclide=True)
        
sps = []
for i in range(0,7):
    sps.append(openmc.StatePoint(f'openmc_simulation_n{i}.h5'))
#    tally = sps[i].get_tally(scores=['flux', 'heating'])
#    flux = tally.get_slice(scores=['flux'])
#    heating = tally.get_slice(scores=['heating'])
#    phi = flux.mean
#    H = heating.mean

#    H *= openmc.data.JOULE_PER_EV
#    vol = 1.24**2
#    f = 174 / H
#    Phi = f * phi / vol
#    print(f'Step: {i}')
#    print(f'Flux: {Phi}')


dfs = []
for sp in sps:
    for rx in xs:
        xs[rx].load_from_statepoint(sp)
    sp.close()
    series = {}
    for rx in xs:
        df = xs[rx].get_pandas_dataframe(xs_type='micro')
        series[rx] = df.set_index('nuclide')['mean']
    dfs.append(DataFrame(series))


/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=1.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=2.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=3.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: IDWarning: Another Surface instance already exists with id=4.
  warn(msg, IDWarning)
/home/ooblack/projects/openmc/openmc/mixin.py:70: ID

In [6]:
dfs

[         (n,gamma)    fission
 nuclide                      
 U234     23.098398   0.497379
 U235     10.506433  48.484407
 U238      0.863002   0.105797
 O16       0.000075   0.000000
 O17       0.000408   0.000000
 U236      8.617116   0.319379,
          (n,gamma)    fission
 nuclide                      
 U234     22.216366   0.498413
 U235     10.466881  48.313321
 U238      0.853190   0.106529
 O16       0.000075   0.000000
 O17       0.000407   0.000000
 U236      9.548907   0.333280,
          (n,gamma)    fission
 nuclide                      
 U234     22.534401   0.500141
 U235     10.453752  48.350339
 U238      0.848853   0.107477
 O16       0.000075   0.000000
 O17       0.000408   0.000000
 U236      8.596615   0.322589,
          (n,gamma)    fission
 nuclide                      
 U234     22.078336   0.495170
 U235     10.605287  49.167829
 U238      0.871416   0.105561
 O16       0.000075   0.000000
 O17       0.000413   0.000000
 U236      8.237984   0.315080,
    

In [7]:
tallies[2].

SyntaxError: invalid syntax (3318269231.py, line 1)